# 🔍 Advanced Clustering: DBSCAN, Hierarchical & UMAP

**Phase 5 Expansion: Методы кластеризации для сложных структур данных**

---

## 🎯 Зачем нужны альтернативы K-Means?

**K-Means limitations:**
- ❌ Требует заранее задать число кластеров K
- ❌ Находит только сферические кластеры
- ❌ Чувствителен к выбросам
- ❌ Не работает с кластерами разной плотности

**Когда K-Means не подходит:**

```
K-Means:          DBSCAN:
   ●●●               ●●●
  ●●●●●             ●●●●●
   ●●●               ●●●
                          ●
      ●●●●●●●●●●         ●●●●●●●●●●
      (1 кластер)        (2 кластера + выброс)
```

---

## 📊 Методы в этом notebook

### 1. **DBSCAN (Density-Based Spatial Clustering)**
- Находит кластеры произвольной формы
- Автоматически определяет число кластеров
- Выделяет выбросы (noise points)
- Параметры: eps (радиус), min_samples

### 2. **Hierarchical Clustering**
- Строит иерархию кластеров (дендрограмма)
- Agglomerative (снизу вверх) или Divisive (сверху вниз)
- Разные метрики связи: single, complete, average, ward
- Можно выбрать число кластеров после построения

### 3. **UMAP (Uniform Manifold Approximation)**
- Dimensionality reduction для визуализации
- Лучше сохраняет глобальную структуру чем t-SNE
- Быстрее t-SNE на больших данных

---


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Clustering
from sklearn.cluster import DBSCAN, AgglomerativeClustering, KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.neighbors import NearestNeighbors

# Hierarchical clustering visualization
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import pdist

# Dimensionality reduction
from sklearn.manifold import TSNE
try:
    import umap
    UMAP_AVAILABLE = True
except ImportError:
    UMAP_AVAILABLE = False
    print("⚠️ UMAP not installed. Install with: pip install umap-learn")

# Datasets
from sklearn.datasets import make_moons, make_circles, make_blobs

plt.style.use('seaborn-v0_8-darkgrid')
np.random.seed(42)

print("✅ Libraries loaded")
if UMAP_AVAILABLE:
    print(f"UMAP version: {umap.__version__}")


## 💾 Часть 1: Synthetic Datasets

Создадим несколько датасетов с разными структурами для демонстрации:
1. **Moons** - два полумесяца (non-convex)
2. **Circles** - концентрические круги
3. **Blobs with varying density** - кластеры разной плотности
4. **Complex real-world** - смесь паттернов

In [ ]:
# Create synthetic datasets
np.random.seed(42)

# 1. Moons - два полумесяца
X_moons, y_moons = make_moons(n_samples=1000, noise=0.05, random_state=42)

# 2. Circles - концентрические круги
X_circles, y_circles = make_circles(n_samples=1000, noise=0.05, factor=0.5, random_state=42)

# 3. Blobs with varying density
# Создаём кластеры разного размера
X_blob1 = np.random.randn(300, 2) * 0.5 + np.array([0, 0])
X_blob2 = np.random.randn(100, 2) * 0.2 + np.array([3, 3])
X_blob3 = np.random.randn(500, 2) * 1.0 + np.array([-3, 3])
X_blobs_varied = np.vstack([X_blob1, X_blob2, X_blob3])
y_blobs_varied = np.array([0]*300 + [1]*100 + [2]*500)

# 4. Complex dataset with noise
X_complex, y_complex = make_blobs(n_samples=800, centers=4, cluster_std=[1.0, 1.5, 0.5, 1.2], random_state=42)
# Add noise points
noise = np.random.uniform(-10, 10, (50, 2))
X_complex = np.vstack([X_complex, noise])
y_complex = np.hstack([y_complex, [-1]*50])  # -1 for noise

datasets = {
    'Moons': (X_moons, y_moons),
    'Circles': (X_circles, y_circles),
    'Varied Density': (X_blobs_varied, y_blobs_varied),
    'Complex + Noise': (X_complex, y_complex)
}

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.flatten()

for idx, (name, (X, y)) in enumerate(datasets.items()):
    scatter = axes[idx].scatter(X[:, 0], X[:, 1], c=y, cmap='viridis', alpha=0.6, s=20)
    axes[idx].set_title(f'{name} (n={len(X)})', fontsize=14, fontweight='bold')
    axes[idx].set_xlabel('Feature 1')
    axes[idx].set_ylabel('Feature 2')

plt.suptitle('Synthetic Datasets for Clustering', fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

print("\n📊 Datasets created:")
for name, (X, y) in datasets.items():
    n_clusters = len(np.unique(y[y >= 0]))
    print(f"  {name}: {len(X)} samples, {n_clusters} true clusters")


---

## 🔬 Часть 2: DBSCAN

### Как работает DBSCAN?

**Ключевые концепции:**

1. **Core point:** точка с ≥ min_samples соседей в радиусе eps
2. **Border point:** не core, но в eps-окрестности core point
3. **Noise point:** ни core, ни border

**Алгоритм:**
1. Для каждой точки найти соседей в радиусе eps
2. Если соседей ≥ min_samples → core point, создать кластер
3. Расширять кластер, добавляя density-reachable точки
4. Точки, не достигнутые ни одним кластером → noise

**Параметры:**
- `eps`: радиус окрестности (важнейший параметр)
- `min_samples`: минимум точек для core point

**Как выбрать eps?**
- k-distance plot (elbow method)
- Домейн knowledge

---


In [ ]:
print("=" * 70)
print("DBSCAN: DENSITY-BASED CLUSTERING")
print("=" * 70)

def find_optimal_eps(X, k=5):
    """
    Find optimal eps using k-distance plot.
    Look for 'elbow' in the sorted k-distances.
    """
    neighbors = NearestNeighbors(n_neighbors=k)
    neighbors.fit(X)
    distances, _ = neighbors.kneighbors(X)
    
    # k-th nearest neighbor distance
    k_distances = distances[:, k-1]
    k_distances = np.sort(k_distances)
    
    return k_distances

# Demonstrate on Moons dataset
print("\n📊 Finding optimal eps for Moons dataset...")

# Scale data
scaler = StandardScaler()
X_moons_scaled = scaler.fit_transform(X_moons)

# K-distance plot
k_distances = find_optimal_eps(X_moons_scaled, k=5)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# K-distance plot
axes[0].plot(k_distances, linewidth=2)
axes[0].set_xlabel('Points (sorted)', fontsize=12)
axes[0].set_ylabel('5-th Nearest Neighbor Distance', fontsize=12)
axes[0].set_title('K-Distance Plot for eps Selection', fontsize=14, fontweight='bold')
axes[0].axhline(y=0.15, color='r', linestyle='--', label='Suggested eps=0.15')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Find elbow (approximate)
eps_suggested = 0.15
print(f"Suggested eps: {eps_suggested}")

# Apply DBSCAN
dbscan = DBSCAN(eps=eps_suggested, min_samples=5)
dbscan_labels = dbscan.fit_predict(X_moons_scaled)

# Results
n_clusters = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)
n_noise = list(dbscan_labels).count(-1)

print(f"\nDBSCAN Results:")
print(f"  Clusters found: {n_clusters}")
print(f"  Noise points: {n_noise} ({n_noise/len(X_moons)*100:.1f}%)")

# Plot results
colors = ['red', 'blue', 'green', 'purple', 'orange']
for label in set(dbscan_labels):
    mask = dbscan_labels == label
    if label == -1:
        axes[1].scatter(X_moons_scaled[mask, 0], X_moons_scaled[mask, 1], 
                       c='gray', marker='x', s=30, label='Noise', alpha=0.5)
    else:
        axes[1].scatter(X_moons_scaled[mask, 0], X_moons_scaled[mask, 1], 
                       c=colors[label % len(colors)], s=30, 
                       label=f'Cluster {label}', alpha=0.6)

axes[1].set_title(f'DBSCAN: {n_clusters} clusters, {n_noise} noise', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Feature 1 (scaled)')
axes[1].set_ylabel('Feature 2 (scaled)')
axes[1].legend()

plt.tight_layout()
plt.show()

print("\n✅ DBSCAN successfully separated the moons!")
print("   K-Means would fail here (non-convex clusters)")


In [ ]:
# Compare DBSCAN vs K-Means on all datasets
print("\n" + "=" * 70)
print("DBSCAN vs K-MEANS COMPARISON")
print("=" * 70)

fig, axes = plt.subplots(4, 3, figsize=(15, 16))

for idx, (name, (X, y_true)) in enumerate(datasets.items()):
    # Scale
    X_scaled = StandardScaler().fit_transform(X)
    
    # True labels
    axes[idx, 0].scatter(X_scaled[:, 0], X_scaled[:, 1], c=y_true, cmap='viridis', s=20, alpha=0.6)
    axes[idx, 0].set_title(f'{name}: True Labels', fontsize=11, fontweight='bold')
    
    # K-Means
    n_true = len(set(y_true[y_true >= 0]))
    kmeans = KMeans(n_clusters=n_true, random_state=42, n_init=10)
    kmeans_labels = kmeans.fit_predict(X_scaled)
    axes[idx, 1].scatter(X_scaled[:, 0], X_scaled[:, 1], c=kmeans_labels, cmap='viridis', s=20, alpha=0.6)
    axes[idx, 1].set_title(f'K-Means (K={n_true})', fontsize=11, fontweight='bold')
    
    # DBSCAN
    # Find good eps for this dataset
    k_dist = find_optimal_eps(X_scaled, k=5)
    eps = np.percentile(k_dist, 90) * 0.5  # heuristic
    
    dbscan = DBSCAN(eps=eps, min_samples=5)
    dbscan_labels = dbscan.fit_predict(X_scaled)
    
    n_clusters = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)
    n_noise = list(dbscan_labels).count(-1)
    
    scatter = axes[idx, 2].scatter(X_scaled[:, 0], X_scaled[:, 1], c=dbscan_labels, cmap='viridis', s=20, alpha=0.6)
    # Mark noise
    noise_mask = dbscan_labels == -1
    if noise_mask.any():
        axes[idx, 2].scatter(X_scaled[noise_mask, 0], X_scaled[noise_mask, 1], 
                           c='red', marker='x', s=30, alpha=0.7)
    axes[idx, 2].set_title(f'DBSCAN ({n_clusters} clusters, {n_noise} noise)', fontsize=11, fontweight='bold')

plt.suptitle('DBSCAN vs K-Means on Different Data Structures', fontsize=14, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

print("\n💡 Key observations:")
print("- Moons/Circles: DBSCAN >> K-Means (non-convex shapes)")
print("- Varied Density: DBSCAN handles different densities better")
print("- Complex + Noise: DBSCAN identifies noise points (red X)")


---

## 🌳 Часть 3: Hierarchical Clustering

### Как работает?

**Agglomerative (bottom-up):**
1. Каждая точка - отдельный кластер
2. Объединяем два ближайших кластера
3. Повторяем пока не останется один кластер
4. Результат - дендрограмма (дерево)

**Linkage methods:**
- **Single:** min distance between clusters (chain effect)
- **Complete:** max distance (compact clusters)
- **Average:** average distance
- **Ward:** minimize variance increase (spherical clusters)

**Преимущества:**
- ✅ Не нужно заранее задавать K
- ✅ Дендрограмма показывает иерархию
- ✅ Можно выбрать K после построения

**Недостатки:**
- ❌ O(n²) память, O(n³) время
- ❌ Не подходит для больших данных

---


In [ ]:
print("=" * 70)
print("HIERARCHICAL CLUSTERING")
print("=" * 70)

# Use Varied Density dataset
X_hier = X_blobs_varied.copy()
X_hier_scaled = StandardScaler().fit_transform(X_hier)

# Compute linkage matrix
print("\nComputing linkage matrices...")
linkage_methods = ['single', 'complete', 'average', 'ward']
linkages = {}

for method in linkage_methods:
    linkages[method] = linkage(X_hier_scaled, method=method)

print("✅ Linkage matrices computed")

# Plot dendrograms
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for idx, method in enumerate(linkage_methods):
    dendrogram(
        linkages[method],
        ax=axes[idx],
        truncate_mode='lastp',
        p=30,  # show last 30 merges
        leaf_rotation=90,
        leaf_font_size=8,
        show_contracted=True
    )
    axes[idx].set_title(f'{method.capitalize()} Linkage', fontsize=14, fontweight='bold')
    axes[idx].set_xlabel('Sample index')
    axes[idx].set_ylabel('Distance')
    
    # Draw cut line for 3 clusters
    if method == 'ward':
        axes[idx].axhline(y=15, color='r', linestyle='--', label='Cut for 3 clusters')
        axes[idx].legend()

plt.suptitle('Dendrograms: Different Linkage Methods', fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

print("\n📊 Linkage comparison:")
print("- Single: chain-like clusters, sensitive to noise")
print("- Complete: compact clusters, sensitive to outliers")
print("- Average: balanced, robust")
print("- Ward: spherical clusters, minimizes variance")


In [ ]:
# Apply Agglomerative Clustering with different linkages
print("\n" + "=" * 70)
print("AGGLOMERATIVE CLUSTERING COMPARISON")
print("=" * 70)

n_clusters = 3  # True number

fig, axes = plt.subplots(2, 3, figsize=(16, 10))

# True labels
axes[0, 0].scatter(X_hier_scaled[:, 0], X_hier_scaled[:, 1], c=y_blobs_varied, cmap='viridis', s=30, alpha=0.6)
axes[0, 0].set_title('True Labels', fontsize=12, fontweight='bold')

# K-Means for comparison
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
kmeans_labels = kmeans.fit_predict(X_hier_scaled)
axes[0, 1].scatter(X_hier_scaled[:, 0], X_hier_scaled[:, 1], c=kmeans_labels, cmap='viridis', s=30, alpha=0.6)
axes[0, 1].set_title('K-Means', fontsize=12, fontweight='bold')

# Different linkages
plot_positions = [(0, 2), (1, 0), (1, 1), (1, 2)]
results = {}

for idx, method in enumerate(linkage_methods):
    agg = AgglomerativeClustering(n_clusters=n_clusters, linkage=method)
    labels = agg.fit_predict(X_hier_scaled)
    results[method] = labels
    
    pos = plot_positions[idx]
    axes[pos].scatter(X_hier_scaled[:, 0], X_hier_scaled[:, 1], c=labels, cmap='viridis', s=30, alpha=0.6)
    
    # Compute silhouette
    sil = silhouette_score(X_hier_scaled, labels)
    axes[pos].set_title(f'{method.capitalize()} (Silhouette: {sil:.3f})', fontsize=12, fontweight='bold')

plt.suptitle('Agglomerative Clustering: Linkage Comparison', fontsize=14, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

print("\n📊 Silhouette Scores:")
for method, labels in results.items():
    sil = silhouette_score(X_hier_scaled, labels)
    print(f"  {method.capitalize():10s}: {sil:.4f}")
print(f"  {'K-Means':10s}: {silhouette_score(X_hier_scaled, kmeans_labels):.4f}")


---

## 🗺️ Часть 4: UMAP Visualization

### UMAP vs t-SNE

**UMAP (Uniform Manifold Approximation and Projection):**
- ✅ Быстрее t-SNE на больших данных
- ✅ Лучше сохраняет глобальную структуру
- ✅ Параметры более интуитивны
- ✅ Может использоваться для transform новых данных

**Ключевые параметры:**
- `n_neighbors`: баланс local vs global structure
- `min_dist`: плотность точек в projection
- `metric`: расстояние (euclidean, cosine, etc.)

---


In [ ]:
print("=" * 70)
print("UMAP vs t-SNE VISUALIZATION")
print("=" * 70)

# Create higher-dimensional dataset for dimensionality reduction demo
from sklearn.datasets import load_digits

# Load digits dataset (8x8 images = 64 dimensions)
digits = load_digits()
X_digits = digits.data
y_digits = digits.target

print(f"\nDigits dataset: {X_digits.shape[0]} samples, {X_digits.shape[1]} dimensions")
print(f"Classes: {len(np.unique(y_digits))} digits (0-9)")

# Scale
X_digits_scaled = StandardScaler().fit_transform(X_digits)

# t-SNE
print("\nRunning t-SNE...")
tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=1000)
X_tsne = tsne.fit_transform(X_digits_scaled)
print("✅ t-SNE done")

# UMAP
if UMAP_AVAILABLE:
    print("Running UMAP...")
    reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, random_state=42)
    X_umap = reducer.fit_transform(X_digits_scaled)
    print("✅ UMAP done")
else:
    X_umap = X_tsne  # fallback
    print("⚠️ UMAP not available, showing t-SNE twice")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# t-SNE
scatter1 = axes[0].scatter(X_tsne[:, 0], X_tsne[:, 1], c=y_digits, cmap='tab10', s=5, alpha=0.7)
axes[0].set_title('t-SNE (perplexity=30)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Dimension 1')
axes[0].set_ylabel('Dimension 2')

# UMAP
scatter2 = axes[1].scatter(X_umap[:, 0], X_umap[:, 1], c=y_digits, cmap='tab10', s=5, alpha=0.7)
axes[1].set_title('UMAP (n_neighbors=15)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Dimension 1')
axes[1].set_ylabel('Dimension 2')

# Colorbar
cbar = plt.colorbar(scatter2, ax=axes[1])
cbar.set_label('Digit')

plt.suptitle('Dimensionality Reduction: 64D → 2D', fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

print("\n💡 Observations:")
print("- Both methods separate digit clusters well")
print("- UMAP typically preserves global structure better")
print("- UMAP is faster on larger datasets")


---

## 📋 Summary & Recommendations

### When to use each method:

| Method | Best for | Avoid when |
|--------|----------|------------|
| **K-Means** | Spherical clusters, known K | Non-convex shapes, outliers |
| **DBSCAN** | Arbitrary shapes, unknown K, noise | Varying densities, high dimensions |
| **Hierarchical** | Small data, need hierarchy, interpretability | Large datasets (>10k) |
| **UMAP** | Visualization, preserving structure | Need exact distances |

### Practical tips:

1. **Always scale data** before clustering
2. **DBSCAN eps:** Use k-distance plot or domain knowledge
3. **Hierarchical linkage:** Ward for spherical, average for general
4. **Evaluation:** Use silhouette, but also visual inspection
5. **High dimensions:** Reduce with PCA/UMAP first

### Key takeaways:

- ✅ **DBSCAN** excels at non-convex clusters and noise detection
- ✅ **Hierarchical** gives interpretable dendrograms
- ✅ **UMAP** is the go-to for visualization
- ✅ **No single best method** - depends on data structure

---
